In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Download the dataset
url='https://datosabiertos.desarrollosocial.gob.ar/dataset/0d022767-9390-486a-bff4-ba53b85d730e/resource/9a951270-60dd-4f21-aa19-4ef1205620bd/download/20220131_info_publica.csv'


path = './'
dataset_file = 'relevamiento-villas-y-asentamientos-AR.csv'

!wget -O path+dataset_file $url


In [ ]:
# Load the dataset.csv as DataFrame
# Convert string -> category

url = path + dataset_file


df = pd.read_csv(url, index_col='id_renabap',
                 dtype={"id_renabap": 'int64',
                        "nombre_barrio": 'category',
                        "provincia": 'category',
                        "departamento": 'category',
                        "localidad": 'category',
                        "cantidad_viviendas_aproximadas": 'int64',
                        "cantidad_familias_aproximadas": 'int64',
                        "decada_de_creacion": 'category',
                        "anio_de_creacion": 'Int64',
                        "energia_electrica": 'category',
                        "efluentes_cloacales": 'category',
                        "agua_corriente": 'category',
                        "cocina": 'category',
                        "calefaccion": 'category',
                        "titulo_propiedad": 'category',
                        "clasificacion_barrio": 'category',
                        "superficie_m2": 'int64'})

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
# Find NA
df.isna().any()


 Vemos que hay valores nulos en la columna **anio de creacion**

In [ ]:
df['anio_de_creacion'].unique()

El año de creacion está registrado solamente a partir de la década del 2010.

In [ ]:
barrios_provincia_count = (df.groupby(['provincia'])['nombre_barrio'].count().reset_index().
                           rename(columns={'nombre_barrio': 'count'}))


barrios_provincia_count

In [ ]:
barrios_provincia_departamento_count = (df.groupby(['provincia', 'departamento'])['nombre_barrio'].count().reset_index().
                                        rename(columns={'nombre_barrio':'count'}))

In [ ]:
barrios_provincia_departamento_count

In [ ]:
viviendas_barrios_populares_prov = (df.groupby(['provincia'])['cantidad_viviendas_aproximadas'].sum().reset_index().
                                    rename(columns={'cantidad_viviendas_aproximadas':'sum'}))

viviendas_barrios_populares_prov


In [ ]:
familias_barrios_populares_prov = (df.groupby(['provincia'])['cantidad_familias_aproximada'].sum().reset_index().
                                   rename(columns={'cantidad_familias_aproximada':'sum'}))

familias_barrios_populares_prov

In [ ]:
alt.Chart(familias_barrios_populares_prov).mark_bar().encode(
    y='provincia',
    x='sum'
)

In [ ]:
alt.Chart(df).mark_bar().encode(
    x='count()',
    y='provincia'
)

# Generación de tablas

## Tabla de cada servicio
### Electricidad

| id_electricidad | tipo de acceso |
| --------------------:  | ---------------- |
|             1          |  23434           |


### Efluentes cloacales

| id_efluentes_cloacales | tipo de acceso |
| ---------------------: | ---------------- |
|             1          |  23434           |


### Agua corriente

### Cocina


### Calefacción

In [ ]:
# Acceso a servicios publicos
servicios = ['energia_electrica', 'calefaccion', 'agua_corriente',
             'efluentes_cloacales', 'cocina']

for nombre_servicio in servicios:
    # Energia electrica
    s = df[[nombre_servicio]].reset_index()
    s.drop(['id_renabap'], axis=1, inplace=True)
    s.drop_duplicates(inplace=True, ignore_index=True)
    # electricidad.drop(['index'], axis=1, inplace=True)
    s.reset_index(drop=True)

    s.index = s.index + 1
    s.index.name = 'id_' + nombre_servicio + '_acceso'

    s.reset_index(inplace=True)

    outfile = nombre_servicio + '.csv'
    s.to_csv(outfile, index=False)




## Tabla de provincias

In [ ]:
provincia = pd.Series(df['provincia'].unique())
provincia.index = provincia.index + 1

provincia.index.name = 'id_provincia'
provincia = provincia.to_frame(name='provincia').reset_index()

provincia

In [ ]:
# Save tables provincias to csv
provincia.to_csv('provincia.csv', index=False)


## Tabla de departamentos

In [ ]:
# Tabla de departamentos
# id_departamento   id_provincia   nombre_departamento

# Extract departamento and provincia from dataframe, drop id_denabap
departamento = df[['departamento', 'provincia']].reset_index()
departamento.drop(['id_renabap'], axis=1, inplace=True)

departamento.reset_index(inplace=True)
departamento.drop(['index'], axis=1, inplace=True)

# Add id_provincia
df2 = pd.merge(departamento, provincia, on='provincia', how='outer')

# Drop duplicate rows
df3 = df2[['departamento', 'provincia', 'id_provincia']].drop_duplicates().reset_index(drop=True)

# Set index name
df3.index = df3.index + 1
df3.index.name = 'id_departamento'
df3.reset_index(inplace=True)

# Drop provincias column
departamento_table = pd.DataFrame(df3[['id_departamento', 'id_provincia', 'departamento']])

# Save table to 'departamento.csv'
departamento_table.to_csv('./departamento.csv', index=False)


departamento_table

## Tabla de localidades

In [ ]:
# Tabla de localidades
# id_localidad   id_departamento   nombre_localidad

# Extract departamento and provincia from dataframe, drop id_denabap
localidad = df[['localidad', 'departamento']].reset_index()
localidad.drop(['id_renabap'], axis=1, inplace=True)

localidad.reset_index(inplace=True)
localidad.drop(['index'], axis=1, inplace=True)

# Add id_provincia
df2 = pd.merge(localidad, departamento_table, on='departamento', how='outer')

# Drop duplicate rows
df3 = df2[['localidad', 'departamento', 'id_departamento']].drop_duplicates().reset_index()

# Set index name
df3.index = df3.index + 1
df3.index.name = 'id_localidad'
df3.reset_index(inplace=True)

# Drop provincias column
localidad_table = pd.DataFrame(df3[['id_localidad', 'id_departamento', 'localidad']])

# Save table to 'departamento.csv'
localidad_table.to_csv('./localidad.csv', index=False)


localidad_table

## Tabla de decadas



In [ ]:
# Tabla de decadas
decada = df['decada_de_creacion'].reset_index()
decada.drop(['id_renabap'], axis=1, inplace=True)
decada.drop_duplicates(inplace=True)
decada.reset_index(inplace=True)
decada = decada.sort_values(by=['decada_de_creacion'])

decada.drop(['index'], axis=1, inplace=True)
decada.reset_index(inplace=True)
decada.index = decada.index +  1
decada.index.name = 'id_decada_creacion'
decada.reset_index(inplace=True)

decada = decada[['id_decada_creacion', 'decada_de_creacion']]

outfile = './decada_de_creacion.csv'

decada.to_csv(outfile, index=False)



In [ ]:
decada

## Tabla de titulo_de_propiedad


In [ ]:
# Tabla de decadas
titulo_de_propiedad = df[['titulo_propiedad']].reset_index()
titulo_de_propiedad.drop(['id_renabap'], axis=1, inplace=True)
titulo_de_propiedad.drop_duplicates(inplace=True)
titulo_de_propiedad.reset_index(inplace=True)

titulo_de_propiedad.drop(['index'], axis=1, inplace=True)
titulo_de_propiedad.reset_index(inplace=True)
titulo_de_propiedad.index = titulo_de_propiedad.index +  1
titulo_de_propiedad.index.name = 'id_titulo_propiedad'
titulo_de_propiedad = titulo_de_propiedad.reset_index()

titulo_de_propiedad = titulo_de_propiedad[['id_titulo_propiedad', 'titulo_propiedad']]

outfile = './titulo_propiedad.csv'

titulo_de_propiedad.to_csv(outfile, index=False)


In [ ]:
titulo_de_propiedad

## Tabla clasificacion_barrio

In [ ]:
# Tabla de decadas
clasificacion_barrio = df[['clasificacion_barrio']].reset_index()
clasificacion_barrio.drop(['id_renabap'], axis=1, inplace=True)
clasificacion_barrio.drop_duplicates(inplace=True)
clasificacion_barrio.reset_index(inplace=True)

clasificacion_barrio.drop(['index'], axis=1, inplace=True)
clasificacion_barrio.reset_index(inplace=True)
clasificacion_barrio.index = clasificacion_barrio.index +  1
clasificacion_barrio.index.name = 'id_clasificacion_barrio'
clasificacion_barrio = clasificacion_barrio.reset_index()

clasificacion_barrio = clasificacion_barrio[['id_clasificacion_barrio', 'clasificacion_barrio']]

outfile = './clasificacion_barrios.csv'

clasificacion_barrio.to_csv(outfile, index=False)


## Tabla de barrios

In [ ]:
barrios_populares = df.copy()
barrios_populares.reset_index(inplace=True)

#tables = ['localidad', 'decada_de_creacion',
#          'energia_electrica', 'efluentes_cloacales', 'agua_corriente',
#          'cocina', 'calefaccion',
#          'titulo_propiedad', 'clasificacion_barrio']
          
localidad = pd.read_csv('localidad.csv')
decada_de_creacion = pd.read_csv('decada_de_creacion.csv')
energia_electrica = pd.read_csv('energia_electrica.csv')
calefaccion = pd.read_csv('calefaccion.csv')
efluentes_cloacales = pd.read_csv('efluentes_cloacales.csv')
cocina = pd.read_csv('cocina.csv')
agua_corriente = pd.read_csv('agua_corriente.csv')






# Add id_provincia
df2 = pd.merge(barrios_populares, localidad, on='localidad', how='outer')

df2.drop_duplicates(inplace=True)
df2.reset_index(inplace=True)
df2.drop('localidad', axis=1, inplace=True)
df2.reset_index(inplace=True)


df3 = pd.merge(df2, energia_electrica, on='energia_electrica', how='outer')
df3.drop_duplicates(inplace=True)

df4 = pd.merge(df3, calefaccion, on='calefaccion', how='outer')
df4.drop_duplicates(inplace=True)

df5 = pd.merge(df4, efluentes_cloacales, on='efluentes_cloacales', how='outer')
df5.drop_duplicates(inplace=True)

df6 = pd.merge(df5, cocina, on='cocina', how='outer')
df6.drop_duplicates(inplace=True)

df7 = pd.merge(df6, agua_corriente, on='agua_corriente', how='outer')
df7.drop_duplicates(inplace=True)

df8 = pd.merge(df7, clasificacion_barrio, on='clasificacion_barrio', how='outer')
df8.drop_duplicates(inplace=True)

df9 = pd.merge(df8, titulo_de_propiedad, on='titulo_propiedad', how='outer')
df9.drop_duplicates(inplace=True)

df10 = pd.merge(df9, decada_de_creacion, on='decada_de_creacion', how='outer')
df10.drop_duplicates(inplace=True)


cols_barrio = ['id_renabap', 'id_localidad', 'id_decada_creacion', 
               'id_energia_electrica_acceso', 'id_efluentes_cloacales_acceso',
               'id_agua_corriente_acceso', 'id_cocina_acceso', 'id_calefaccion_acceso',
               'id_clasificacion_barrio', 'id_titulo_propiedad',
               'nombre_barrio', 'cantidad_viviendas_aproximadas' ]




In [ ]:
df2.drop_duplicates()


In [ ]:
df10['id_renabap'].unique()

In [ ]:
foreign_keys = ['id_localidad', 'id_decada_de_creacion',
                'id_electricidad', 'id_calefaccion',
                'id_agua_corriente', 'id_efluentes_cloacales',
                'id_cocina']
  
tablas = ['localidad', 'decada_de_creacion', 'energia_electrica',
          'calefaccion', 'agua_corriente', 'id_efluentes_cloacales',
          'id_cocina']
          
    
for t in tablas:
    filename = t + '.csv'
    table_df = pd.read_csv(filename)
    print (table_df) 
    
    


In [ ]:
barrios_table_attr = ['nombre_barrio', 'cantidad_viviendas_aproximadas', 'cantidad_familias_aproximada',
                     'titulo_propiedad', 'clasificacion_barrio', 'superficie_m2']

df[barrios_table_attr]
